In [1]:
import pandas as pd
import xgboost
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

In [2]:

def utc_ts():
    return datetime.datetime.utcnow(
        ).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%SZ')
utc_ts()

'2020-06-08T142456Z'

#### The data


In [3]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [84]:
def prepare_data(tripsdf, stationsdf):
    
    # Step 1, merge w/ stationsdf to get neighborhood data
    mdf = tripsdf[['start station name', 'end station name', 'gender']
            ].merge(stationsdf[['station_name', 'neighborhood']], 
                    left_on='start station name',
                    right_on='station_name'
                   ).rename(columns={'neighborhood': 'start_neighborhood'}
                           ).merge(stationsdf[['station_name', 'neighborhood']],
                                  left_on='end station name',
                                   right_on='station_name'
                                  ).rename(columns={'neighborhood': 'end_neighborhood'})
    
    neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
    
    X, y = (mdf[['start_neighborhood', 'gender']].values, 
            np.array(mdf['end_neighborhood'].tolist()))
    return X, y
    
def preprocess(X, y, neighborhoods):
    # Initially assuming labeled=True
    labeled = True
    genders = [0, 1, 2]

    enc = OneHotEncoder(handle_unknown='error', 
                        categories=[neighborhoods, genders])
    enc.fit(X)
    X_transformed = enc.transform(X)
    
    le = LabelEncoder()
    le.fit(neighborhoods)
    
    y_enc = le.transform(y)    
    
    return X, X_transformed, y, enc, le, y_enc
    
class FooFlassifier():
    def __init__(self, stationsdf):
        self.stationsdf = stationsdf
        self.neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
        self.workdir = make_work_dir()
    def fit(self, X, y):
        # preproc
        (X, X_transformed, y, 
             self.one_hot_enc, self.le,
             y_enc) = preprocess(X, y, self.neighborhoods)
        joblib.dump({'le': self.le,
                    'one_hot_enc': self.one_hot_enc},
                   f'{self.workdir}/artifacts.joblib')
        self.clf = xgboost.XGBClassifier(num_class=75)
        self.clf.fit(X_transformed, y_enc)
        
        self.labels = list(range(len(self.le.classes_)))
        
    def score(self, X, y_true):
        X_transformed = self.one_hot_enc.transform(X)
        y_true_enc = self.le.transform(y_true)
        
        y_prob = self.clf.predict_proba(X_transformed)
        return log_loss(y_true_enc, y_prob, labels=self.labels)

    def get_params(self, deep):
        return {}
def make_work_dir():
    ts = utc_ts()
    workdir = f'/opt/program/artifacts/{ts}' 
    os.mkdir(workdir)
    return workdir

#X, y = mdf[['start_neighborhood', 'gender']].values, np.array(mdf['end_neighborhood'].tolist())

#clf = FooFlassifier(stationsdf) 

In [66]:
from joblib import dump, load
import joblib

In [50]:
X, y = prepare_data(tripsdf, stationsdf)

In [51]:
X.shape, y.shape

((843416, 2), (843416,))

In [85]:
model = FooFlassifier(stationsdf)
ipdb.runcall(model.fit, X[:100, :], y[:100])

> <ipython-input-84-35411273e1d6>(46)fit()
     45              self.one_hot_enc, self.le,
---> 46              y_enc) = preprocess(X, y, self.neighborhoods)
     47         joblib.dump({'le': self.le,

ipdb> c


XGBoostError: [14:47:39] src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (7500 vs. 100) : labels are not correctly providedpreds.size=7500, label.size=100
Stack trace:
  [bt] (0) /opt/conda/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x45) [0x7f6dda4333f5]
  [bt] (1) /opt/conda/lib/libxgboost.so(xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0x248) [0x7f6dda4ef838]
  [bt] (2) /opt/conda/lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x28d) [0x7f6dda44d04d]
  [bt] (3) /opt/conda/lib/libxgboost.so(XGBoosterUpdateOneIter+0x48) [0x7f6dda5d6798]
  [bt] (4) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x69dd) [0x7f6e0a7eb9dd]
  [bt] (5) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x6067) [0x7f6e0a7eb067]
  [bt] (

In [81]:
len(neighborhoods)

75

In [71]:
with ipdb.launch_ipdb_on_exception():
    clf.score(X[:2], y[:2])
    

ValueError('The number of classes in labels is different from that in y_pred. Classes found in labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23\n 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47\n 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71\n 72 73 74]')


> /opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py(2291)log_loss()
   2290                              'from that in y_pred. Classes found in '
-> 2291                              'labels: {0}'.format(lb.classes_))
   2292 

ipdb> u
> <ipython-input-65-f7c59b48ea24>(60)score()
     59         y_prob = self.clf.predict_proba(X_transformed)
---> 60         return log_loss(y_true_enc, y_prob, labels=self.labels)
     61 

ipdb> p y_prob.shape
(2, 2)
ipdb> y_prob
array([[0.9978419 , 0.00215811],
       [0.9978419 , 0.00215811]], dtype=float32)
ipdb> p y_true_enc.shape
(2,)
ipdb> p y_true_enc
array([60, 60])
ipdb> p self.labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
ipdb> c


In [80]:
clf.clf.n_classes_

1

In [ ]:
# Note in https://stackoverflow.com/questions/39386966/multiclass-classification-in-xgboost-python#39387627 
# about clf.classes_ and clf._n_classes of an xgboost clf classfier,
# being set in the clf.fit() method
# But I wonder if I can set the classes prior to fit , so I dont have to 
# necessarily make sure all the classes are in the training set prior.
# If that is the only way then I will do that though (balanced training set )

In [83]:
# A hint at this in https://github.com/dmlc/xgboost/issues/5449 actually using a num_class 
#    parameter. 
# Although I can set it, I cannot peak at n_classes_ 
# xgboost.XGBClassifier(num_class=75).n_classes_
# AttributeError: 'XGBClassifier' object has no attribute 'n_classes_'

AttributeError: 'XGBClassifier' object has no attribute 'n_classes_'

In [ ]:
# 2020-06-11 0202Z
# here, https://stackoverflow.com/questions/37845920/cross-validation-on-xgbclassifier-for-multiclass-classification-in-python
# there's another interesting approach, showing the use of `xgb.cv()` in place of sklearn's
# cross validation which I have been using, 
# `from sklearn.model_selection import cross_val_score`
# - The xgb.cv() func takes `metrics={'mlogloss'}` as a param, as opposed to ,
#   manually using the sklearn log_loss as I had. 

# Objective functions.
# other than 'mlogloss', there is also 'multi:softprob' 
# Not sure if those are different names for the same thing.

In [26]:
X_transformed.shape, X.shape, y.shape

((843416, 78), (843416, 2), (843416,))

In [27]:
y[:5]

array(['Stuyvesant Town', 'Stuyvesant Town', 'Stuyvesant Town',
       'Stuyvesant Town', 'Stuyvesant Town'], dtype='<U35')

In [29]:
from sklearn.preprocessing import LabelEncoder
# Get data..
neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
le = LabelEncoder()
le.fit(neighborhoods)
y_enc = le.transform(y)
#Xdm = xgboost.DMatrix(X_transformed[:5,:])
#from xgboost import XGBClassifier
clf = xgboost.XGBClassifier()

In [33]:
len(neighborhoods)

75

In [30]:
X_transformed.shape, y_enc.shape

((843416, 78), (843416,))

In [31]:
%time clf.fit(X_transformed, y_enc)

CPU times: user 13min 58s, sys: 2.19 s, total: 14min
Wall time: 13min 59s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [44]:
workdir = make_work_dir()
joblib.dump({'clf': clf}, f'{workdir}/bundle.joblib')

['/opt/program/2020-06-10T025852Z/bundle.joblib']

In [89]:
clf = joblib.load('/opt/program/artifacts/2020-06-10T025852Z/bundle.joblib')

In [92]:
len(clf['clf'].classes_)

54

In [97]:
from collections import Counter
sorted(list(dict(Counter(y_enc)).keys()))

[0,
 1,
 2,
 4,
 5,
 7,
 8,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 21,
 23,
 24,
 25,
 26,
 27,
 30,
 32,
 33,
 34,
 36,
 37,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 57,
 58,
 60,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 71,
 72,
 73]

In [34]:
clf.predict_proba(X_transformed[:2])

array([[0.01552367, 0.01610189, 0.00264415, 0.00248007, 0.01490377,
        0.00548938, 0.00194296, 0.02104487, 0.08323692, 0.00932526,
        0.01069923, 0.00389162, 0.0011591 , 0.0095946 , 0.00404946,
        0.0107293 , 0.01842718, 0.0157086 , 0.01104296, 0.00274902,
        0.02098426, 0.0144077 , 0.02674809, 0.0424823 , 0.00737197,
        0.0533537 , 0.00576095, 0.01814625, 0.00236372, 0.02293371,
        0.01015841, 0.00756694, 0.06421325, 0.1442597 , 0.08239184,
        0.03230955, 0.00267815, 0.00657854, 0.00776298, 0.00174899,
        0.00284085, 0.00655629, 0.00576877, 0.00975472, 0.00180184,
        0.0414222 , 0.02034637, 0.0035512 , 0.00375667, 0.02123439,
        0.0110102 , 0.00067188, 0.02583351, 0.01048607],
       [0.01552367, 0.01610189, 0.00264415, 0.00248007, 0.01490377,
        0.00548938, 0.00194296, 0.02104487, 0.08323692, 0.00932526,
        0.01069923, 0.00389162, 0.0011591 , 0.0095946 , 0.00404946,
        0.0107293 , 0.01842718, 0.0157086 , 0.01104296, 0.0

In [18]:
neighborhoods = ['Midtown East', 'Stuyvesant Town',]; gender = [0,1,2]
enc = OneHotEncoder(handle_unknown='ignore', categories=[neighborhoods, gender])
X = np.array([['Midtown East', 1],
             ['Midtown East',2],
             ['Stuyvesant Town',0]])
X2 = np.array([['foo', 1],
             ['Midtown East',2],
             ['Stuyvesant Town',0]])
enc.fit(X)


OneHotEncoder(categories=[['Midtown East', 'Stuyvesant Town'], [0, 1, 2]],
              drop=None, dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              sparse=True)

In [19]:
enc.transform(X2).toarray()

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.]])

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
le = preprocessing.LabelEncoder()
neigh